Group Level Analysis

In [12]:
import pandas as pd
import nipype.pipeline as pe
from nipype.interfaces import DataSink
import nipype.interfaces.fsl as fsl

In [2]:
# Suggested by paper
problematic_subjects = ["sub-10998", "sub-10159", "sub-70055", "sub-70022", "sub-70048", "sub-10680"]

control_subjects = pd.read_csv("control_subjects_ids.txt", header=None)
adhd_subjects = pd.read_csv("adhd_subjects_ids.txt", header=None)

# remove problematic subjects from adhd and control subjects
control_subjects = control_subjects[~control_subjects[0].isin(problematic_subjects)]
adhd_subjects = adhd_subjects[~adhd_subjects[0].isin(problematic_subjects)]

# add control column and adhd column to both adhd and control subjects
control_subjects['control'] = 1
control_subjects['adhd'] = 0

adhd_subjects['control'] = 0
adhd_subjects['adhd'] = 1

subjects = pd.concat([control_subjects, adhd_subjects])

print(subjects.head())
print(subjects.tail())


           0  control  adhd
1  sub-11106        1     0
2  sub-11044        1     0
3  sub-10524        1     0
4  sub-10708        1     0
5  sub-10429        1     0
            0  control  adhd
30  sub-70079        0     1
31  sub-70080        0     1
32  sub-70081        0     1
33  sub-70083        0     1
34  sub-70086        0     1


In [3]:
contrasts = [['group_mean', 'T', ['control', 'adhd'], [1, -1]]] # FSL compatible
regressor = {"control": subjects['control'].to_list(), "adhd": subjects['adhd'].to_list()}

In [4]:
from pathlib import Path
import nibabel as nib
import numpy as np
# get mask files from all subjects
mask_files = []
masks = np.empty((65,77,49, len(subjects)))
current_mask = None
for i, subject_id in enumerate(subjects[0]):
    mask_file = Path(f"derivatives/{subject_id}/func/{subject_id}_task-scap_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz")
    if mask_file.exists():
        mask = nib.load(mask_file.absolute()) # type: ignore
        masks[:,:,:,i] = mask.get_fdata() # type: ignore
        current_mask = mask

mask = masks.mean(axis=3)
mask = np.where(mask > 0.8, 1, 0)
mask_image = nib.Nifti1Image(mask, current_mask.affine, current_mask.header)

Path("working_group").mkdir(exist_ok=True)
mask_file = Path("working_group/group_mask.nii.gz").absolute()
mask_image.to_filename(mask_file)

In [17]:
# get feat dirs
feat_dirs = []
for subject_id in subjects[0]:
    # feat_dir = Path(f"results/{subject_id}/scap.feat")
    feat_dir = Path(f"derivatives/task/{subject_id}/scap.feat")
    if feat_dir.exists():
        feat_dirs.append(feat_dir)

print("Loaded", len(feat_dirs), "feat dirs")

Loaded 64 feat dirs


In [18]:

# 19 Linear Up Load
# 21 Linear Down Delay

SELECT_CONTRAST = 1

# Get cope files
cope_files = [x.absolute() / "stats" / f"cope{SELECT_CONTRAST}.nii.gz" for x in feat_dirs if (x / "stats" / f"cope{SELECT_CONTRAST}.nii.gz").exists()]
print("Loaded", len(cope_files), "cope files")

varcopes = [x.absolute() / "stats" / f"varcope{SELECT_CONTRAST}.nii.gz" for x in feat_dirs if (x / "stats" / f"varcope{SELECT_CONTRAST}.nii.gz").exists()]
print("Loaded", len(varcopes), "varcope files")

print(cope_files[0])

workflow = pe.Workflow(name="group_level_analysis")

merge_cope = pe.Node(fsl.Merge(dimension="t"), name="merge_cope")
merge_cope.inputs.in_files = cope_files

merge_varcope = pe.Node(fsl.Merge(dimension="t"), name="merge_varcope")
merge_varcope.inputs.in_files = varcopes

model = pe.Node(fsl.MultipleRegressDesign(), name="model")
model.inputs.contrasts = contrasts
model.inputs.regressors = regressor
model.inputs.groups = [1 for _ in range(len(subjects))]


flameo = pe.Node(fsl.FLAMEO(
    mask_file=mask_file,
    run_mode="flame1",
), name="flameo")

workflow.connect(model, 'design_mat', flameo, 'design_file')
workflow.connect(model, 'design_con', flameo, 't_con_file')
workflow.connect(merge_cope, 'merged_file', flameo, 'cope_file')
workflow.connect(merge_varcope, 'merged_file', flameo, 'var_cope_file')
workflow.connect(model, 'design_grp', flameo, 'cov_split_file')

smooth_estimate = pe.Node(fsl.SmoothEstimate(mask_file=mask_file), name="smooth_estimate")

workflow.connect(flameo, 'zstats', smooth_estimate, 'zstat_file')

cluster = pe.Node(fsl.Cluster(), name="cluster")
workflow.connect(smooth_estimate,'dlh', cluster, 'dlh')
workflow.connect(smooth_estimate, 'volume', cluster, 'volume')

cluster.inputs.connectivity = 26
cluster.inputs.threshold = 2.3
cluster.inputs.pthreshold = 0.05
cluster.inputs.out_threshold_file = True
cluster.inputs.out_index_file = True
cluster.inputs.out_localmax_txt_file = True
workflow.connect(flameo, "zstats", cluster, "in_file")

ztopval = pe.Node(fsl.ImageMaths(op_string="-ztop", suffix="_pval"), name="ztopval")
workflow.connect(flameo, "zstats", ztopval, "in_file")

group_results_dir = Path(f"group_results/scap/contrast{SELECT_CONTRAST}")
group_results_dir.mkdir(parents=True, exist_ok=True)

sinker = pe.Node(DataSink(), name="sinker")
sinker.inputs.base_directory = str(group_results_dir.absolute())
sinker.inputs.substitutions = [('_cope_id', 'contrast'),('_maths_', '_reversed_')]

workflow.connect(flameo, 'zstats', sinker, 'stats')
workflow.connect(cluster, 'threshold_file', sinker, 'stats.@thr')
workflow.connect(cluster, 'index_file', sinker, 'stats.@index')
workflow.connect(cluster, 'localmax_txt_file', sinker, 'stats.@localmax')

zstats_reversed = pe.Node(fsl.BinaryMaths(operation="mul", operation_value=-1), name="zstats_reversed")
workflow.connect(flameo, 'zstats', zstats_reversed, 'in_file')

cluster_reversed = cluster.clone("cluster_reversed")
workflow.connect(zstats_reversed, 'out_file', cluster_reversed, 'in_file')


workflow.run()












Loaded 64 cope files
Loaded 64 varcope files
/home/rongfei/WorkSpace/consortium/derivatives/task/sub-11106/scap.feat/stats/cope1.nii.gz
250920-15:46:07,749 nipype.workflow INFO:
	 Workflow group_level_analysis settings: ['check', 'execution', 'logging', 'monitoring']
250920-15:46:07,754 nipype.workflow INFO:
	 Running serially.
250920-15:46:07,754 nipype.workflow INFO:
	 [Node] Setting-up "group_level_analysis.model" in "/tmp/tmp_opnun13/group_level_analysis/model".
250920-15:46:07,756 nipype.workflow INFO:
	 [Node] Executing "model" <nipype.interfaces.fsl.model.MultipleRegressDesign>
250920-15:46:07,758 nipype.workflow INFO:
	 [Node] Finished "model", elapsed time 0.000523s.
250920-15:46:07,759 nipype.workflow INFO:
	 [Node] Setting-up "group_level_analysis.merge_cope" in "/tmp/tmplxlj6u8r/group_level_analysis/merge_cope".
250920-15:46:07,761 nipype.workflow INFO:
	 [Node] Executing "merge_cope" <nipype.interfaces.fsl.utils.Merge>
250920-15:46:08,516 nipype.workflow INFO:
	 [Node] Fin